In [ ]:


from sklearn.cross_validation import train_test_split
import pandas as pd
import xgboost as xgb

random_seed = 1225

#set data path
train_x_csv = "C:/Users/tianjiayang/loan_predicting/data/train_x.csv"
train_y_csv = "C:/Users/tianjiayang/loan_predicting/data/train_y.csv"
test_x_csv = "C:/Users/tianjiayang/loan_predicting/data/test_x.csv"
features_type_csv = "C:/Users/tianjiayang/loan_predicting/data/features_type.csv"

#load data
train_x = pd.read_csv(train_x_csv)
train_y = pd.read_csv(train_y_csv)
train_xy = pd.merge(train_x,train_y,on='uid')

test = pd.read_csv(test_x_csv)
test_uid = test.uid
test_x = test.drop(['uid'],axis=1)

#dictionary {feature:type}
features_type = pd.read_csv(features_type_csv)
features_type.index = features_type.feature
features_type = features_type.drop('feature',axis=1)
features_type = features_type.to_dict()['type']


feature_info = {}
features = list(train_x.columns)
features.remove('uid')

for feature in features:
    max_ = train_x[feature].max()
    min_ = train_x[feature].min()
    n_null = len(train_x[train_x[feature]<0])  #number of null
    n_gt1w = len(train_x[train_x[feature]>10000])  #greater than 10000
    feature_info[feature] = [min_,max_,n_null,n_gt1w]

#see how many neg/pos sample
print ("neg:{0},pos:{1}".format(len(train_xy[train_xy.y==0]),len(train_xy[train_xy.y==1])))


#split train set,generate train,val,test set
train_xy = train_xy.drop(['uid'],axis=1)
# 20% for validation
train,val = train_test_split(train_xy, test_size = 0.2,random_state=1)#random_state is of big influence for val-auc
y = train.y
X = train.drop(['y'],axis=1)
val_y = val.y
val_X = val.drop(['y'],axis=1)



#xgboost start here
dtest = xgb.DMatrix(test_x)
dval = xgb.DMatrix(val_X,label=val_y)
dtrain = xgb.DMatrix(X, label=y)
params={
    #gbtree,gblinear,dart
	'booster':'gbtree',
    #
	'objective': 'rank:pairwise',
    #100轮后指标不上升提前终止
    
	'early_stopping_rounds':100,
    #Control the balance of positive and negative weights, useful for unbalanced classes. 
	'scale_pos_weight': 1500.0/13458.0,
    #
    'eval_metric': 'auc',
    #Minimum loss reduction required to make a further partition on a leaf node of the tree. 
    #The larger gamma is, the more conservative the algorithm will be.
	'gamma':0.1,#0.2 is ok
    #Maximum depth of a tree.
	'max_depth':8,
    #L2 reg
	'lambda':550,
    #Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees.
    'subsample':0.7,
    #Subsample ratio of columns when constructing each tree.
    'colsample_bytree':0.3,
    #Min sum of instance weight needed in a child. If the tree partition step results in a leaf node with the sum of instance weig
    'min_child_weight':2.5, 
    #learning rate
    'eta': 0.005,
	'seed':random_seed,
	'nthread':7
    }

watchlist  = [(dtrain,'train'),(dval,'val')]#The early stopping is based on last set in the evallist
model = xgb.train(params,dtrain,num_boost_round=15000,evals=watchlist)
#Save the model to a file.
model.save_model('./model/xgb.model')
print ("best best_ntree_limit{}".format(model.best_ntree_limit))  

#predict test set (from the best iteration)
test_y = model.predict(dtest,ntree_limit=model.best_ntree_limit)
test_result = pd.DataFrame(columns=["uid","score"])
test_result.uid = test_uid
test_result.score = test_y
test_result.to_csv("xgb.csv",index=None,encoding='utf-8')  #remember to edit xgb.csv , add ""


#save feature score and feature information:  feature,score,min,max,n_null,n_gt1w
feature_score = model.get_fscore()
for key in feature_score:
    feature_score[key] = [feature_score[key]]+feature_info[key]+[features_type[key]]

feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1},{2},{3},{4},{5},{6}\n".format(key,value[0],value[1],value[2],value[3],value[4],value[5]))

with open('feature_score.csv','w') as f:
    f.writelines("feature,score,min,max,n_null,n_gt1w\n")
    f.writelines(fs)





C:\Users\tianjiayang\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


neg:1542,pos:13458
[22:31:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 212 extra nodes, 4 pruned nodes, max_depth=8
[0]	train-auc:0.693546	val-auc:0.619132
[22:31:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 0 pruned nodes, max_depth=8
[1]	train-auc:0.731444	val-auc:0.629084
[22:31:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 4 pruned nodes, max_depth=8
[2]	train-auc:0.745915	val-auc:0.645508
[22:31:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 232 extra nodes, 0 pruned nodes, max_depth=8
[3]	train-auc:0.754412	val-auc:0.641297
[22:31:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 218 extra nodes, 8 pruned nodes, max_depth=8
[4]	train-auc:0.756923	val-auc:0.640359
[22:31:33] C:\Users\A

[22:31:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=8
[43]	train-auc:0.783743	val-auc:0.663423
[22:31:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 8 pruned nodes, max_depth=8
[44]	train-auc:0.784167	val-auc:0.663181
[22:31:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 228 extra nodes, 4 pruned nodes, max_depth=8
[45]	train-auc:0.783971	val-auc:0.663129
[22:31:37] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 238 extra nodes, 4 pruned nodes, max_depth=8
[46]	train-auc:0.784456	val-auc:0.66334
[22:31:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 6 pruned nodes, max_depth=8
[47]	train-auc:0.784639	val-auc:0.663554
[22:31:38] C:\Users\Administrator\De

[22:31:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 8 pruned nodes, max_depth=8
[86]	train-auc:0.791916	val-auc:0.66629
[22:31:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=8
[87]	train-auc:0.791698	val-auc:0.666505
[22:31:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 2 pruned nodes, max_depth=8
[88]	train-auc:0.792354	val-auc:0.667043
[22:31:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 198 extra nodes, 4 pruned nodes, max_depth=8
[89]	train-auc:0.792115	val-auc:0.666683
[22:31:42] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 214 extra nodes, 6 pruned nodes, max_depth=8
[90]	train-auc:0.792267	val-auc:0.666573
[22:31:42] C:\Users\Administrator\De

[22:31:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 230 extra nodes, 10 pruned nodes, max_depth=8
[129]	train-auc:0.797955	val-auc:0.669951
[22:31:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 230 extra nodes, 2 pruned nodes, max_depth=8
[130]	train-auc:0.798106	val-auc:0.669954
[22:31:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 254 extra nodes, 4 pruned nodes, max_depth=8
[131]	train-auc:0.798271	val-auc:0.670219
[22:31:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 224 extra nodes, 6 pruned nodes, max_depth=8
[132]	train-auc:0.798315	val-auc:0.670066
[22:31:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 246 extra nodes, 10 pruned nodes, max_depth=8
[133]	train-auc:0.798579	val-auc:0.670036
[22:31:47] C:\Users\Administ

...

[22:44:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 94 pruned nodes, max_depth=8
[6608]	train-auc:0.99957	val-auc:0.718765
[22:44:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 94 pruned nodes, max_depth=8
[6609]	train-auc:0.99957	val-auc:0.718776
[22:44:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 84 pruned nodes, max_depth=8
[6610]	train-auc:0.99957	val-auc:0.718752
[22:44:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 82 pruned nodes, max_depth=8
[6611]	train-auc:0.999571	val-auc:0.718765
[22:44:47] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 90 pruned nodes, max_depth=8
[6612]	train-auc:0.999572	val-auc:0.718766
[22:44:47] C:\Users\Adm

[22:44:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 80 pruned nodes, max_depth=8
[6650]	train-auc:0.999583	val-auc:0.718874
[22:44:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 80 pruned nodes, max_depth=8
[6651]	train-auc:0.999584	val-auc:0.718893
[22:44:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 76 pruned nodes, max_depth=8
[6652]	train-auc:0.999584	val-auc:0.718914
[22:44:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 106 pruned nodes, max_depth=8
[6653]	train-auc:0.999585	val-auc:0.718898
[22:44:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 180 extra nodes, 88 pruned nodes, max_depth=8
[6654]	train-auc:0.999585	val-auc:0.718899
[22:44:52] C:\Users

[22:44:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 86 pruned nodes, max_depth=8
[6692]	train-auc:0.999598	val-auc:0.718872
[22:44:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 84 pruned nodes, max_depth=8
[6693]	train-auc:0.999598	val-auc:0.718874
[22:44:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 100 pruned nodes, max_depth=8
[6694]	train-auc:0.999599	val-auc:0.718887
[22:44:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 154 extra nodes, 82 pruned nodes, max_depth=8
[6695]	train-auc:0.999599	val-auc:0.71887
[22:44:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 84 pruned nodes, max_depth=8
[6696]	train-auc:0.9996	val-auc:0.718872
[22:44:57] C:\Users\Ad

[22:45:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 104 pruned nodes, max_depth=8
[6734]	train-auc:0.999613	val-auc:0.718895
[22:45:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 72 pruned nodes, max_depth=8
[6735]	train-auc:0.999613	val-auc:0.718876
[22:45:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 152 extra nodes, 104 pruned nodes, max_depth=8
[6736]	train-auc:0.999613	val-auc:0.718888
[22:45:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 98 pruned nodes, max_depth=8
[6737]	train-auc:0.999614	val-auc:0.718873
[22:45:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 90 pruned nodes, max_depth=8
[6738]	train-auc:0.999614	val-auc:0.718874
[22:45:02] C:\User

[22:45:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 84 pruned nodes, max_depth=8
[6776]	train-auc:0.999626	val-auc:0.718928
[22:45:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 82 pruned nodes, max_depth=8
[6777]	train-auc:0.999627	val-auc:0.718924
[22:45:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 138 extra nodes, 80 pruned nodes, max_depth=8
[6778]	train-auc:0.999627	val-auc:0.718937
[22:45:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 64 pruned nodes, max_depth=8
[6779]	train-auc:0.999627	val-auc:0.718939
[22:45:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 80 pruned nodes, max_depth=8
[6780]	train-auc:0.999628	val-auc:0.718919
[22:45:07] C:\Users\